## Coupon Purchase Prediction

In [ ]:
import pandas as pd

df = pd.read_csv('user_list.csv')
print(df)

                  REG_DATE SEX_ID  AGE WITHDRAW_DATE PREF_NAME  \
0      2012-03-28 14:14:18      f   25           NaN       NaN   
1      2011-05-18 00:41:48      f   34           NaN       東京都   
2      2011-06-13 16:36:58      m   41           NaN       愛知県   
3      2012-02-08 12:56:15      m   25           NaN       NaN   
4      2011-05-22 23:43:56      m   62           NaN      神奈川県   
...                    ...    ...  ...           ...       ...   
22868  2011-12-12 15:42:56      f   24           NaN       NaN   
22869  2011-08-10 00:49:55      m   41           NaN       京都府   
22870  2012-04-05 12:24:23      f   35           NaN       NaN   
22871  2011-02-20 10:34:22      f   59           NaN       NaN   
22872  2011-02-24 15:43:18      f   38           NaN       埼玉県   

                           USER_ID_hash  
0      d9dca3cb44bab12ba313eaa681f663eb  
1      560574a339f1b25e57b0221e486907ed  
2      e66ae91b978b3229f8fd858c80615b73  
3      43fc18f32eafb05713ec02935e2c2825

We eliminate users whose prefecture is unknown

In [ ]:
datas = df.dropna(subset=["PREF_NAME"])
print(datas)

                  REG_DATE SEX_ID  AGE WITHDRAW_DATE PREF_NAME  \
1      2011-05-18 00:41:48      f   34           NaN       東京都   
2      2011-06-13 16:36:58      m   41           NaN       愛知県   
4      2011-05-22 23:43:56      m   62           NaN      神奈川県   
5      2011-05-27 16:17:19      f   50           NaN       広島県   
6      2011-08-24 23:30:24      m   44           NaN       埼玉県   
...                    ...    ...  ...           ...       ...   
22860  2010-11-16 11:45:48      f   29           NaN       宮城県   
22861  2011-08-16 01:43:50      f   35           NaN       東京都   
22864  2010-12-17 21:17:39      m   42           NaN       長崎県   
22869  2011-08-10 00:49:55      m   41           NaN       京都府   
22872  2011-02-24 15:43:18      f   38           NaN       埼玉県   

                           USER_ID_hash  
1      560574a339f1b25e57b0221e486907ed  
2      e66ae91b978b3229f8fd858c80615b73  
4      dc6df8aa860f8db0d710ce9d4839840f  
5      f430e8302c1fd0915bce0c068ecb2f82

We eliminate the users who have unsubscribed

In [ ]:
datas = datas[datas.isnull().any(1)]
print(datas)

                  REG_DATE SEX_ID  AGE WITHDRAW_DATE PREF_NAME  \
1      2011-05-18 00:41:48      f   34           NaN       東京都   
2      2011-06-13 16:36:58      m   41           NaN       愛知県   
4      2011-05-22 23:43:56      m   62           NaN      神奈川県   
5      2011-05-27 16:17:19      f   50           NaN       広島県   
6      2011-08-24 23:30:24      m   44           NaN       埼玉県   
...                    ...    ...  ...           ...       ...   
22860  2010-11-16 11:45:48      f   29           NaN       宮城県   
22861  2011-08-16 01:43:50      f   35           NaN       東京都   
22864  2010-12-17 21:17:39      m   42           NaN       長崎県   
22869  2011-08-10 00:49:55      m   41           NaN       京都府   
22872  2011-02-24 15:43:18      f   38           NaN       埼玉県   

                           USER_ID_hash  
1      560574a339f1b25e57b0221e486907ed  
2      e66ae91b978b3229f8fd858c80615b73  
4      dc6df8aa860f8db0d710ce9d4839840f  
5      f430e8302c1fd0915bce0c068ecb2f82

We reset the indices the final data will be filtered.

In [ ]:
datas.reset_index(drop=True)

We save the filtered data

In [ ]:
from google.colab import files
datas.to_csv('user_list_filtrado.csv',index=False)
files.download('user_list_filtrado.csv')

**FEATURES: AGE, SEX, PREFECTURE**

**TARGET: Type of Coupon bought**

Now we will create unit vectors of size equal to the number of coupons available(one for each coupon)

We load the file with existing coupons coupons_list_train.csv

In [ ]:
import pandas as pd

df = pd.read_csv('coupon_list_train.csv')
print(df)

      CAPSULE_TEXT GENRE_NAME  PRICE_RATE  CATALOG_PRICE  DISCOUNT_PRICE  \
0              グルメ        グルメ          50           3000            1500   
1              グルメ        グルメ          51           2080            1000   
2              グルメ        グルメ          50           7000            3500   
3              グルメ        グルメ          50           3000            1500   
4              グルメ        グルメ          50           2000            1000   
...            ...        ...         ...            ...             ...   
16598        ヘアサロン      ヘアサロン          82          11025            1980   
16599          エステ        エステ          78           9450            2000   
16600          グルメ        グルメ          53           4500            2100   
16601       ネイル・アイ     ネイル・アイ          52           6300            2980   
16602         レッスン       レッスン          73          73500              19   

                  DISPFROM              DISPEND  DISPPERIOD   VALIDFROM  \
0      2011-

We create a dictionary to be able to access the vector corresponding to each coupon

In [ ]:
import numpy as np

sizes=df["COUPON_ID_hash"].size
vectores=np.identity(sizes) 
dictionary_coupons={} 

for i in range(sizes):
  dictionary_coupons[df.loc[i,"COUPON_ID_hash"]]=vectores[i] 


In [ ]:
print(dictionary_coupons)

{'6b263844241eea98c5a97f1335ea82af': array([1., 0., 0., ..., 0., 0., 0.]), 'cc031f250e8bad1e24060263b9fc0ddd': array([0., 1., 0., ..., 0., 0., 0.]), 'ba5e9b7453ca52ff711635a5d2e8102d': array([0., 0., 1., ..., 0., 0., 0.]), '3e1ffbedca3569f9e8032d401e8cb4e6': array([0., 0., 0., ..., 0., 0., 0.]), '782934b6c815b4030ea204eef7d4a734': array([0., 0., 0., ..., 0., 0., 0.]), '439db4336f84d4e4a13290fc0d7e6c4e': array([0., 0., 0., ..., 0., 0., 0.]), 'c9144a97ed61cb778414b758c704d828': array([0., 0., 0., ..., 0., 0., 0.]), '2b00d8dfc1a9fcf58e5014cbfd4a1e48': array([0., 0., 0., ..., 0., 0., 0.]), '74e3a12659d38ef20378f156cfeaf204': array([0., 0., 0., ..., 0., 0., 0.]), '8a0ab8c5dd5014df8b4860233238ed44': array([0., 0., 0., ..., 0., 0., 0.]), '5544cd0309f920f50f7e0d7e7d6fa69f': array([0., 0., 0., ..., 0., 0., 0.]), '6eda8088f6394d430f1a1e0abf619f6a': array([0., 0., 0., ..., 0., 0., 0.]), '305d6620db89f174eae5071749397446': array([0., 0., 0., ..., 0., 0., 0.]), '784d04cd55f5eaf24b5b2a361dca722d': a

Now we will create unit vectors of size equal to the number of coupons available.

In [ ]:
import pandas as pd

df2 = pd.read_csv('coupon_area_train.csv')
print(df2)

       SMALL_AREA_NAME PREF_NAME                    COUPON_ID_hash
0                   埼玉       埼玉県  6b263844241eea98c5a97f1335ea82af
1                   千葉       千葉県  cc031f250e8bad1e24060263b9fc0ddd
2                   千葉       千葉県  ba5e9b7453ca52ff711635a5d2e8102d
3                   千葉       千葉県  3e1ffbedca3569f9e8032d401e8cb4e6
4                   千葉       千葉県  782934b6c815b4030ea204eef7d4a734
...                ...       ...                               ...
138180              新潟       新潟県  fe334f1a5e0dc54eca76b9dd9559661b
138181  新宿・高田馬場・中野・吉祥寺       東京都  8dc11b257cb1c5b98e7b7c66d9b2b779
138182      渋谷・青山・自由が丘       東京都  b15e2d7619c3fd9c6be567c971bdf4f4
138183     銀座・新橋・東京・上野       東京都  e1141a4b2798450f36e40090659c0b07
138184  新宿・高田馬場・中野・吉祥寺       東京都  79303de03e3af7105cded55960d62ac3

[138185 rows x 3 columns]


we create a dictionary to be able to access the vector corresponding to each picture.

In [ ]:
import numpy as np

sizes=df2["PREF_NAME"].size
vectores=np.identity(sizes) 
dictionary_prefectures={} 

for i in range(sizes):
  dictionary_prefectures[df2.loc[i,"PREF_NAME"]]=vectores[i] 
  

The number of prefectures is so large that more Ram than available in COLAB would be needed, so this time we will ignore this.

We create the set of training features X={{age1,sex1}, {age2,sex2}}

0 = male

1= female

We load the file coupon_detail_train.csv to select the pointer user_id hash of each example and see the age, gender using filtred file user_filtredo.csv

Note: do not consider where user cases are not found in the database

In [ ]:
import pandas as pd

cp = pd.read_csv('coupon_detail_train.csv')
print(cp)

        ITEM_COUNT               I_DATE SMALL_AREA_NAME  \
0                1  2012-03-28 15:06:06              兵庫   
1                1  2011-07-04 23:52:54     銀座・新橋・東京・上野   
2                1  2011-07-16 00:52:49       恵比寿・目黒・品川   
3                1  2011-07-16 00:54:53       恵比寿・目黒・品川   
4                1  2011-07-16 00:55:52       恵比寿・目黒・品川   
...            ...                  ...             ...   
168991           1  2012-02-07 12:14:50       池袋・神楽坂・赤羽   
168992           1  2012-02-28 15:43:21     銀座・新橋・東京・上野   
168993           1  2012-03-19 12:11:16     銀座・新橋・東京・上野   
168994           2  2012-04-12 12:27:34  新宿・高田馬場・中野・吉祥寺   
168995           1  2012-05-09 12:12:26  新宿・高田馬場・中野・吉祥寺   

                         PURCHASEID_hash                      USER_ID_hash  \
0       c820a8882374a4e472f0984a8825893f  d9dca3cb44bab12ba313eaa681f663eb   
1       1b4eb2435421ede98c8931c42e8220ec  560574a339f1b25e57b0221e486907ed   
2       36b5f9ba46c44b65587d0b16f2e4c77f  560574a339f1b25

In [ ]:
import pandas as pd

users = pd.read_csv('user_list_filtrado.csv')
print(users)

                  REG_DATE SEX_ID  AGE  WITHDRAW_DATE PREF_NAME  \
0      2011-05-18 00:41:48      f   34            NaN       東京都   
1      2011-06-13 16:36:58      m   41            NaN       愛知県   
2      2011-05-22 23:43:56      m   62            NaN      神奈川県   
3      2011-05-27 16:17:19      f   50            NaN       広島県   
4      2011-08-24 23:30:24      m   44            NaN       埼玉県   
...                    ...    ...  ...            ...       ...   
15240  2010-11-16 11:45:48      f   29            NaN       宮城県   
15241  2011-08-16 01:43:50      f   35            NaN       東京都   
15242  2010-12-17 21:17:39      m   42            NaN       長崎県   
15243  2011-08-10 00:49:55      m   41            NaN       京都府   
15244  2011-02-24 15:43:18      f   38            NaN       埼玉県   

                           USER_ID_hash  
0      560574a339f1b25e57b0221e486907ed  
1      e66ae91b978b3229f8fd858c80615b73  
2      dc6df8aa860f8db0d710ce9d4839840f  
3      f430e8302c1fd0915bce

In [ ]:
#cp.loc[1,"COUPON_ID_hash"]
#index=users.USER_ID_hash[users.USER_ID_hash == cp.loc[2,"USER_ID_hash"]].index.tolist()
#index
#cp["COUPON_ID_hash"].size 

X=[] 
training_set_size=0 
for i in range(int(round(0.3*cp["COUPON_ID_hash"].size))):
  index=users.USER_ID_hash[users.USER_ID_hash == cp.loc[i,"USER_ID_hash"]].index.tolist() 
  if index!=[]:
    training_set_size+=1
    x1=int(users.loc[index,"AGE"])
    if str(users.loc[index,"SEX_ID"]).split()[1]=='f':
      x2=1
    else:
      x2=0
      
    X.append([x1,x2])

We create the set of target vectors from he training set T = {{0,0,1}, {0,1,0}}

To do this we load the coupon_detail_train.csv file to select the ITEM_COUNT and COUPON_ID_hash pointers.

We use the dictionary_coupons and we access it using the coupon_id_hash pointer as the key. The result is the target vector of the example in turn, which gives the probability of buying a coupon given the training features.

In [ ]:
import pandas as pd

cp = pd.read_csv('coupon_detail_train.csv')
print(cp)

        ITEM_COUNT               I_DATE SMALL_AREA_NAME  \
0                1  2012-03-28 15:06:06              兵庫   
1                1  2011-07-04 23:52:54     銀座・新橋・東京・上野   
2                1  2011-07-16 00:52:49       恵比寿・目黒・品川   
3                1  2011-07-16 00:54:53       恵比寿・目黒・品川   
4                1  2011-07-16 00:55:52       恵比寿・目黒・品川   
...            ...                  ...             ...   
168991           1  2012-02-07 12:14:50       池袋・神楽坂・赤羽   
168992           1  2012-02-28 15:43:21     銀座・新橋・東京・上野   
168993           1  2012-03-19 12:11:16     銀座・新橋・東京・上野   
168994           2  2012-04-12 12:27:34  新宿・高田馬場・中野・吉祥寺   
168995           1  2012-05-09 12:12:26  新宿・高田馬場・中野・吉祥寺   

                         PURCHASEID_hash                      USER_ID_hash  \
0       c820a8882374a4e472f0984a8825893f  d9dca3cb44bab12ba313eaa681f663eb   
1       1b4eb2435421ede98c8931c42e8220ec  560574a339f1b25e57b0221e486907ed   
2       36b5f9ba46c44b65587d0b16f2e4c77f  560574a339f1b25

In [ ]:
#dictionary_coupons["34c48f84026e08355dc3bd19b427f09a"]

T=[] #vector target

for i in range(int(round(0.3*cp["COUPON_ID_hash"].size))):
  index=users.USER_ID_hash[users.USER_ID_hash == cp.loc[i,"USER_ID_hash"]].index.tolist() 
  if index!=[]: 
    t=dictionary_coupons[cp.loc[i,"COUPON_ID_hash"]]
    T.append(t)

We design a neural network with the appropriate chracteristics for the training set

Whose output is a softmax multiple (returns a vector of probabilities)

In [ ]:
from keras import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import PReLU
from keras.layers import ELU
from keras.layers import Dropout
from keras.utils.vis_utils import plot_model

sizes=df["COUPON_ID_hash"].size


def build_regressor():
    regressor = Sequential()
    regressor.add(Dense(units=2, input_shape = (2,)))
    regressor.add(ELU(alpha=1)) 
    regressor.add(Dense(units=int(round((sizes)/3))))
    regressor.add(ELU(alpha=1))
    regressor.add(Dropout(0.25)) 
    regressor.add(Dense(units=sizes,activation='softmax')) 
    regressor.compile(optimizer='Nadam', loss='categorical_crossentrophy',  metrics=['accuracy'])
    return regressor

build_regressor().summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 2)                 6         
                                                                 
 elu_3 (ELU)                 (None, 2)                 0         
                                                                 
 dense_4 (Dense)             (None, 5534)              16602     
                                                                 
 elu_4 (ELU)                 (None, 5534)              0         
                                                                 
 dropout_1 (Dropout)         (None, 5534)              0         
                                                                 
 dense_5 (Dense)             (None, 16603)             91897605  
                                                                 
Total params: 91,914,213
Trainable params: 91,914,213


In [ ]:
iterations = 20 
from keras.wrappers.scikit_learn import KerasRegressor
regressor = KerasRegressor(build_fn=build_regressor,batch_size=20,epochs=iterations,validation_split=0.2, shuffle=True, verbose=2)

In [ ]:
import time 
tic = time.time()

results = regressor.fit(np.array(X), np.array(T))
print("Time processing (secs):", time.time()-tic)

Our model measures each user's affinity for coupons. A high affinity for some coupon indicates that the user is more likely to buy some ticket.

In [ ]:
import random

age=random.randint(1,100)
sex=np.random.choice(2) 
if sex==0:
  sex_str="M"
else:
  sex_str="F"

arr=regressor.predict(np.array([[age,sex]]))
sum=0
for i in range(1,11):
  sum+=np.partition(arr.flatten(), -2)[-i]
if sum>0.1:
  guarantee="YES"
else:
  guarantee="NO"


max_prob=np.amax(arr)
result = np.where(arr == np.amax(arr))
sizes=df["COUPON_ID_hash"].size 
vector=np.zeros(sizes)
#vector[result[1][0]]=1

 
print("AGE: ", age)
print("SEX: ", sex_str)
print("Coupon most probable to be sold: ", [k for k,v in dictionary_coupons.items() if np.array_equal(v,vector)])
print('Prob. of selling the coupon most probable to be sold: ', round(max_prob*100,3),"%")
print('Prob. of selling any of the 10 coupons most probable to be sold: ', round(sum*100,3),"%")
print("Almost Ensured sell?", guarantee)

1/1 - 0s - 30ms/epoch - 30ms/step
AGE:  21
SEX:  M
Coupon most probable to be sold:  []
Prob. of selling the coupon most probable to be sold:  0.006 %
Prob. of selling any of the 10 coupons most probable to be sold:  0.063 %
Almost Ensured sell? NO


In [ ]:
from prettytable import PrettyTable
num_random_users=25 
x = PrettyTable()

x.field_names = ["AGE", "SEX", "Most salable coupon for the given user", "Prob. Most salable coupon", "Prob. 10 Most salable coupons", "The user will buy a Coupon?"]

for i in range(num_random_users):
  age=random.randint(1,100)
  sex=np.random.choice(2)
  if sex==0:
    sex_str="M"
  else:
    sex_str="F"
    
  arr=regressor.predict(np.array([[age,sex]]))
  sum=0
  for i in range(1,11): 
    sum+=np.partition(arr.flatten(), -2)[-i]
  if sum>0.1:
    guarantee="YES"
  else:
    guarantee="NO"
 
  max_prob=np.amax(arr)
  result = np.where(arr == np.amax(arr))
  sizes=df["COUPON_ID_hash"].size 
  vector=np.zeros(sizes)
  #vector[result[1][0]]=1

  x.add_row([age, sex_str, [k for k,v in dictionary_coupons.items() if np.array_equal(v,vector)], str(round(max_prob*100,3))+" %",str(round(sum*100,3))+" %",guarantee])

print(x)

1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 17ms/epoch - 17ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
+-----+-----+----------------------------------------+---------------------------+-------------------------------+-----------------------------+
| AGE